In [ ]:
import requests
import json

In [ ]:
url = "https://bankaccountdata.gocardless.com/api/v2/token/new/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
with open('keys.json', 'rb') as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
response = requests.post(url, headers=headers, data=json.dumps(data))

# Stampa il risultato
print(response.status_code)
print(response.json())  # mostra il token se la richiesta ha successo

In [ ]:
ACCESS_TOKEN = response.json()["access"]

In [ ]:
url = "https://bankaccountdata.gocardless.com/api/v2/institutions/"
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}"  # <-- sostituisci con il token reale
}
params = {
    "country": "it"
}

response = requests.get(url, headers=headers, params=params)

# Stampa la risposta
print(response.status_code)
print(response.json())  # mostra la lista delle istituzioni

In [ ]:
institutions = response.json()

In [ ]:
for inst in institutions:
    print(inst['name'])

In [ ]:
my_bank = [ii for ii in institutions if ii['name'] == 'CREDEM Retail'][0]

In [ ]:
my_bank

In [ ]:
response = requests.post(
    url="https://bankaccountdata.gocardless.com/api/v2/requisitions/", 
    headers={
        "accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"  # <-- Sostituisci con il token reale
    }, 
    json={
        "redirect": "http://www.yourwebpage.com",
        "institution_id": my_bank['id'],
        "reference": "111",
        "user_language": "IT"
    }
)

# Verifica la risposta
print(response.status_code)
print(response.json())


In [ ]:
requitision = response.json()

In [ ]:
response = requests.get(
    url=f"https://bankaccountdata.gocardless.com/api/v2/requisitions/{requitision['id']}/",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"  # <-- sostituisci con il token reale
    }
)
# Verifica la risposta
print(response.status_code)
print(response.json())


In [ ]:
user_profile = response.json()

In [ ]:
response = requests.get(
    url=f"https://bankaccountdata.gocardless.com/api/v2/accounts/{user_profile['accounts'][0]}/balances/" ,
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"  # <-- sostituisci con il token reale
    }
)
# Verifica la risposta
print(response.status_code)
print(response.json())

In [ ]:
balance = response.json()

In [ ]:
balance